In [ ]:
!pip install langdetect

In [ ]:
!pip install Sastrawi

## Importing Packages

Import all essential libraries and frameworks for comprehensive text preprocessing, including pandas for data manipulation, numpy for numerical operations, matplotlib/seaborn for visualization, NLTK for natural language processing, scikit-learn for text vectorization, and specialized libraries for language detection and text cleaning operations.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

import warnings
warnings.filterwarnings('ignore')

## Loading Data

In this section you are required to load data from local and then extract it to the storage directory.


In [ ]:
dataframe = pd.read_csv('traveloka_assessment.csv')

In [ ]:
dataframe.head(10)

In [ ]:
dataframe['reviewCreatedVersion'].drop_duplicates()

## Data Preprocessing

Conduct initial data exploration and preparation steps including checking data types, identifying missing values, examining data distribution, and understanding the overall structure of the dataset. This foundational analysis guides the subsequent preprocessing strategy.

In [ ]:
dataframe.info()

In [ ]:
dataframe.dtypes

In [ ]:
print(dataframe.shape)
print("\n Check Is Null")
print(dataframe.isnull().sum())
print("\n Check Duplicated")
print(dataframe.duplicated().sum())

In [ ]:
datacheck = ['content', 'score', 'thumbsUpCount']
dataframe[datacheck].head(10)

## Clean Data (Text Normalization)

Perform the first phase of data cleaning focusing on basic text normalization. This includes handling missing values, removing duplicates, standardizing text encoding, and addressing any structural inconsistencies in the dataset that could affect downstream processing.

In [ ]:
# Deteksi bahasa untuk setiap review
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

In [ ]:
dataframe['language'] = dataframe['content'].apply(detect_language)

In [ ]:
dataframe['language'].value_counts()

In [ ]:
# remove that languange non 'id'
dataframe = dataframe[dataframe['language'] == 'id'].reset_index(drop=True)

In [ ]:
df = dataframe.copy()
df = df[['content', 'score']]
df.head(10)

In [ ]:
# create labeling function
def label_score(score):
    if score >= 4:
        return 'positive'
    elif score == 3:
        return 'neutral'
    else:
        return 'negative'

In [ ]:
df['sentiment'] = df['score'].apply(label_score)
df['sentiment'].value_counts()

## Cleaning Data (basic version + Stopwords + Stemming)

Implement comprehensive text cleaning pipeline that includes:

- <b>Basic Cleaning:</b> Remove special characters, URLs, email addresses, and unwanted symbols
- <b>Stopwords Removal:</b> Eliminate common words that don't contribute to sentiment or meaning
- <b>Stemming:</b> Reduce words to their root form using algorithms like Porter Stemmer to normalize word variations and reduce dimensionality

Key Features:

- <b>Case normalization </b> (converting to lowercase)
- <b>Punctuation removal and handling</b>
- <b>Number and digit processing</b>
- <b>HTML tag removal if present</b>
- <b>Whitespace normalization</b>
- <b>Language-specific stopword filtering</b>
- <b>Root word extraction through stemming algorithms</b>

In [ ]:
# initialize stemmer and stopword remover
stemmer = StemmerFactory().create_stemmer()
stop_factory = StopWordRemoverFactory()
stopwords = set(stop_factory.get_stop_words())

In [ ]:
# cleaning function
def clean_text(text):
    # Remove url
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Hilangkan angka dan tanda baca
    text = re.sub(r'[^a-z\s]', '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # manual tokenize
    tokens = text.split()
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords]
    # stemming
    text = ' '.join([stemmer.stem(word) for word in tokens])
    return text

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
df['clean_content'] = df['content'].progress_apply(clean_text)

In [ ]:
df['clean_content'].head(10)

## Save Clean Data

In [ ]:
df.to_csv('traveloka_clean.csv', index=False)

In [ ]:
cleandf = pd.read_csv('traveloka_clean.csv')

In [ ]:
cleandf.head()